In [ ]:
# 1. پیاده‌سازی MLP با Numpy
# یک لایه مخفی

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def load_data_numpy(path):
    """
    Load the CSV, extract features and target, 
    standardize features, and split into train/test.
    """
    df = pd.read_csv(path)
    feat_cols = ['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms',
                 'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']
    X = df[feat_cols].values.astype(np.float32)
    # standardize X
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    # target = house price (continuous)
    y = df['House_Price'].values.astype(np.float32)
    # train/test split
    return train_test_split(X, y, test_size=0.3, random_state=42)

def mlp_train_numpy(X_train, y_train, hidden_size=10, lr=1e-4, epochs=200):
    """
    A simple MLP with one hidden layer (tanh activation),
    trained by vanilla SGD on MSE loss.
    Returns weight matrices W1,b1, W2,b2.
    """
    n_samples, n_features = X_train.shape
    # initialize weights small random, biases zero
    W1 = np.random.randn(n_features, hidden_size).astype(np.float32) * 0.01
    b1 = np.zeros(hidden_size, dtype=np.float32)
    W2 = np.random.randn(hidden_size, 1).astype(np.float32) * 0.01
    b2 = np.zeros(1, dtype=np.float32)
    
    for epoch in range(epochs):
        # Shuffle for each epoch (optional)
        idx = np.random.permutation(n_samples)
        for i in idx:
            x = X_train[i]          # shape: (n_features,)
            y_true = y_train[i]     # scalar

            # ----- forward pass -----
            z1 = x.dot(W1) + b1     # shape: (hidden_size,)
            a1 = np.tanh(z1)        # tanh activation
            z2 = a1.dot(W2) + b2    # shape: (1,)
            y_pred = z2[0]          # scalar

            # ----- compute gradients (MSE loss) -----
            # dL/dy_pred
            d_loss = 2.0 * (y_pred - y_true)

            # output layer gradients
            grad_W2 = a1.reshape(-1,1) * d_loss   # (hidden_size,1)
            grad_b2 = d_loss                      # scalar

            # backprop into hidden layer
            # dL/da1 = W2 * d_loss
            d_a1 = (W2.flatten()) * d_loss        # (hidden_size,)
            # d(a1)/dz1 for tanh = 1 - a1^2
            d_z1 = d_a1 * (1 - a1**2)             # (hidden_size,)

            # hidden layer gradients
            grad_W1 = np.outer(x, d_z1)           # (n_features, hidden_size)
            grad_b1 = d_z1                        # (hidden_size,)

            # ----- parameter update -----
            W2 -= lr * grad_W2
            b2 -= lr * grad_b2
            W1 -= lr * grad_W1
            b1 -= lr * grad_b1

        # optionally print loss every 50 epochs
        if (epoch+1) % 50 == 0:
            # compute train MSE
            y_train_pred = mlp_predict_numpy(X_train, W1, b1, W2, b2)
            mse = np.mean((y_train_pred - y_train)**2)
            print(f"[Epoch {epoch+1}] train MSE = {mse:.2f}")

    return W1, b1, W2, b2

def mlp_predict_numpy(X, W1, b1, W2, b2):
    """
    Forward pass through the trained MLP.
    Returns a 1D array of predictions.
    """
    z1 = X.dot(W1) + b1            # (n_samples, hidden_size)
    a1 = np.tanh(z1)               # (n_samples, hidden_size)
    z2 = a1.dot(W2) + b2           # (n_samples, 1)
    return z2.flatten()

if __name__ == "__main__":
    # load data
    X_train, X_test, y_train, y_test = load_data_numpy('house_price_regression_dataset.csv')
    # train
    W1, b1, W2, b2 = mlp_train_numpy(X_train, y_train,
                                     hidden_size=10, lr=1e-4, epochs=200)
    # predict & eval
    y_pred = mlp_predict_numpy(X_test, W1, b1, W2, b2)
    mse_test = np.mean((y_pred - y_test)**2)
    print(f"Numpy MLP Test MSE: {mse_test:.2f}")
    # df = pd.DataFrame({X_test, y_pred})
    # print(tabulate(df, headers=feat_cols))

[Epoch 50] train MSE = 24449265664.00
[Epoch 100] train MSE = 21956933632.00
[Epoch 150] train MSE = 21833105408.00
[Epoch 200] train MSE = 21827110912.00
Numpy MLP Test MSE: 20655226880.00


TypeError: unhashable type: 'numpy.ndarray'